In [2]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from tqdm import tqdm
import dspy
import re
from sisyphus.utils.helper_functions import get_plain_articledb

e:\anaconda\envs\sisyphus\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# this is modify branch
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
)
article_db = get_plain_articledb('heas_batch_1')

def embed_one(article):
    docs = article_db.get(article)
    with ThreadPoolExecutor(max_workers=20) as worker:
        futures = [worker.submit(vector_store.add_documents, [doc]) for doc in docs]
        for future in futures:
            future.result()

K = 3
QUERY_SYN = """Experimental procedures describing the synthesis and processing of HEAs materials, including methods such as melting, casting, rolling, annealing, heat treatment, or other fabrication techniques. Details often include specific temperatures (e.g., °C), durations (e.g., hours, minutes), atmospheric conditions (e.g., argon, vacuum), mechanical deformation (e.g., rolling reduction)."""
QUERY_MECHANICAL = """Mechanical properties of high entropy alloys, stress-strain curves, yield strength, ultimate tensile strength, tensile strain, elongation, alloy composition, alloying effects on strength, ductility, engineering stress-strain behavior."""
QUERY_PHASE = """Phase characterization of high entropy alloys (common phases include FCC, BCC, HCP, etc.), microstructure analysis, crystal structures, phase transitions, XRD patterns, lattice parameters, grain morphology, recrystallization, secondary phases, alloying effects on phases, defect structures, and phase stability."""

def match_subtitles(docs, pattern):
    sub_titles = list(set([doc.metadata["sub_titles"] for doc in docs]))
    target_titles = []
    for title in sub_titles:
        if pattern.search(title):
            target_titles.append(title)
    return target_titles

syn_pattern = re.compile(r'(experiment)|(preparation)|(method)', re.I)
res_pattern = re.compile(r'result', re.I)
test_pattern = re.compile(r'strain\srate', re.I)
def retrieve(vector_store, article, query, sub_titles):
    if sub_titles is None:
        filter_ = {"source": article}
    else:
        filter_ = {"$and":[{
            "sub_titles": {
                "$in": sub_titles
            }},
            {"source": article
        }]}
    return vector_store.similarity_search(
        query,
        k=K,
        filter=filter_
    )


In [4]:
import os

articles = os.listdir('articles_processed')[:20]
articles

['10.1002&sol;adem.201900587.html',
 '10.1002&sol;adem.202200523.html',
 '10.1002&sol;adma.201807142.html',
 '10.1002&sol;mawe.202300263.html',
 '10.1016&sol;j.actamat.2020.08.043.html',
 '10.1016&sol;j.actamat.2021.117571.html',
 '10.1016&sol;j.actamat.2022.118516.html',
 '10.1016&sol;j.actamat.2024.120498.html',
 '10.1016&sol;j.actamat.2024.120614.html',
 '10.1016&sol;j.addma.2020.101202.html',
 '10.1016&sol;j.commatsci.2022.111888.html',
 '10.1016&sol;j.corsci.2021.110073.html',
 '10.1016&sol;j.corsci.2022.110357.html',
 '10.1016&sol;j.corsci.2023.111422.html',
 '10.1016&sol;j.corsci.2023.111708.html',
 '10.1016&sol;j.ijhydene.2019.04.280.html',
 '10.1016&sol;j.ijhydene.2020.11.154.html',
 '10.1016&sol;j.ijrmhm.2023.106163.html',
 '10.1016&sol;j.ijrmhm.2023.106225.html',
 '10.1016&sol;j.intermet.2013.03.018.html']

In [5]:
for article in tqdm(articles):
    vector_store.add_documents(article_db.get(article))

100%|██████████| 20/20 [00:53<00:00,  2.70s/it]


In [6]:
from typing import Literal

lm = dspy.LM('openai/gpt-4o-mini', cache=False, temperature=0)
dspy.configure(lm=lm)

def get_target_para(article, query, pattern, classifier, class_):
    docs = article_db.get(article)
    sub_titles = match_subtitles(docs, pattern) or None
    candidates = retrieve(vector_store, article, query, sub_titles)
    final = []
    with ThreadPoolExecutor(5) as worker:
        futures = [worker.submit(classifier, paragraph=candidate.page_content) for candidate in candidates]
        future_doc = {future: candidate for future, candidate in zip(futures, candidates)}
        for future in as_completed(futures):
            if future.result().topic == class_:
                final.append(future_doc[future])
    return final

class ClassifySyn(dspy.Signature):
    """assign topic to paragraphs of HEAs(high entropy alloys) papers. The topics include synthesis, characterization, and others.
    Note: a qualified synthesis paragraph should include the synthesis and processing of materials, including methods such as melting, casting, rolling, annealing, heat treatment.be very strict about your decision."""
    paragraph: str = dspy.InputField()
    topic: Literal['synthesis', 'characterization', 'others'] = dspy.OutputField()

classifier_syn = dspy.ChainOfThought(signature=ClassifySyn)

class ClassifyMech(dspy.Signature):
    """assign topic to paragraphs of HEAs(high entropy alloys) papers. The topics include tensile/compressive with value, tensile/compressive without value, characterization or others.
    Note: a qualified tensile/compressive with value paragraph should explicitly mention at least one value related to yield strength, ultimate yield strength, elongation, or compressive strain, otherwise it should be classified as tensile/compressive without value.Be very strict about your decision."""
    paragraph: str = dspy.InputField()
    topic: Literal['tensile/compressive with value', 'tensile/compressive without value', 'characterization', 'others'] = dspy.OutputField()
classifier_mech = dspy.ChainOfThought(signature=ClassifyMech)

class ClassifyPha(dspy.Signature):
    """assign topic to paragraphs of HEAs(high entropy alloys) papers. The topics include characterization_phase, characterization_others, or others.
    Note: a qualified phase charaterization paragraph should include the descripion of XRD patterns indicating the crystal structures such as FCC, BCC, HCP or other structures."""
    paragraph: str = dspy.InputField()
    topic: Literal['characterization_phase', 'characterization_others', 'others'] = dspy.OutputField()
classifier_pha = dspy.ChainOfThought(signature=ClassifyPha)

class ClassifyTable(dspy.Signature):
    """assign labels to tables of HEAs(high entropy alloys) papers. The labels include tensile_exp, compress_exp, tensile_theoretical, compress_theoretical or others."""
    paragraph: str = dspy.InputField()
    topic: Literal['tensile_exp', 'compress_exp', 'tensile_theoretical', 'compress_theoretical', 'others'] = dspy.OutputField(desc="the label given to the table. If it mentions data was calculated, you should label it theoretical. Please be very strict about your decision.")
classifier_table = dspy.ChainOfThought(signature=ClassifyTable)

def get_tables(docs):
    tables = [doc for doc in docs if doc.metadata["sub_titles"] == "table"]
    final = []
    with ThreadPoolExecutor(5) as worker:
        futures = [worker.submit(classifier_table, paragraph=table.page_content) for table in tables]
        future_doc = {future: table for future, table in zip(futures, tables)}
        for future in as_completed(futures):
            if future.result().topic in ['tensile_exp', 'compress_exp']:
                final.append(future_doc[future])
    return final

In [9]:
class ExtractSteps(dspy.Signature):
    """Extract synthesis steps from a HEAs experimental section. Formated as below:
    Material: [material]
    Synthesis methods
    Fabrication: [fabrication] (methods like induction melting, additive manufacturing etc.)
    Thermo-mechanical processings (if any)
    [processing_1]: [processing_1_parameters] (e.g. 900 °C, 6 h)
    [processing_2]: [processing_2_parameters]
    ...

    Note:
    Only extract synthesis steps of HEAs material, do not include any characterization steps.
    For those only have melting-casting processings, do not include any thermo-mechanical processings.
    If the author indicates different material results from different processing conditions, include the corresponding material after each condition"""
    abstract: str = dspy.InputField(desc='The abstract of the paper')
    paragraph: str = dspy.InputField(desc='The experimental section of the paper')
    steps: str = dspy.OutputField()
steps_extractor = dspy.Predict(signature=ExtractSteps)

one_shot_example_1 = """Ingots of Co60Ni40 alloy and Co20Cr40Ni40 MEA were fabricated by vacuum arc-melting of pure metals (purity > 99.9 wt.%) under an inert gas (high-purity argon) atmosphere. After melting, they were cooled in a water-cooled copper mold and flipped and re-melted five times to improve compositional homogeneity. Subsequently, the ingots were cold-rolled to a 30% reduction in thickness and homogenized at 1100 °C for 24 h. Then, the homogenized plate of the Co60Ni40 alloy and the Co20Cr40Ni40 MEA were further cold-rolled to a 92% reduction in thickness and annealed at 750 °C for 120 s and 850 °C for 3.6 ks, respectively. These processes yielded fully-recrystallized microstructures of FCC single phase having similar mean grain sizes of about 3 μm (including annealing twins) in the two alloys."""
one_shot_example_2 = """The master alloy of AlCoCrFeNi2.1 was prepared from commercially pure elements (Al, Co, Ni: 99.8 wt %; Cr, Fe: 99.5-99.5 wt %). The experimented material was received in vacuum arc remelted condition, the chemical composition of which is listed in Table 1 ."""
answer = ["""Material: Co60Ni40 alloy and Co20Cr40Ni40 MEA
Synthesis methods
Fabrication: Vacuum arc-melting
Thermo-mechanical processings
Cold-rolling: 30% reduction in thickness
Homogenization: 1100 °C, 24 h
Cold-rolling: 92% reduction in thickness
Annealing:
- 750 °C, 120 s (Co60Ni40 alloy)
- 850 °C, 3.6 ks (Co20Cr40Ni40 MEA)""",
"Material: AlCoCrFeNi2.1\nSynthesis methods\nFabrication: Vacuum arc remelting\nThermo-mechanical processings\nNone"]
examples = [dspy.Example(abstract=abst, paragraph=para, steps=s).with_inputs('paragraph') for abst, para, s in zip(['', ''], [one_shot_example_1, one_shot_example_2], answer)]
compiler = dspy.LabeledFewShot()
two_shot_steps_extractor = compiler.compile(steps_extractor, trainset=examples)

class ExtractPhases(dspy.Signature):
    """Extract HEAs material crystal system from phase characterization paragraph. Formated as below,
    [HEAs formula]: [phases] (choose from FCC, BCC, HCP, B2, Laves, L12 or others)
    ...(if multiple materials)
    Note: if there are multiple phases, separate them with commas"""
    abstract: str = dspy.InputField(desc='The abstract of the paper')
    paragraph: str = dspy.InputField(desc='Phase characterization section of the paper')
    phases: str = dspy.OutputField()
phases_extractor = dspy.Predict(signature=ExtractPhases)

In [7]:
from pydantic import BaseModel, Field
from typing import Literal, List
from typing import Optional
class AlloyRecord(BaseModel):
    composition: str = Field(description='The nominal composition of the alloy, ensure the validity of the formula, e.g. CoCrFeNi')
    phase: Optional[str] = Field(description='The phase of the alloy, such as FCC, BCC, HCP etc. If there are multiple phases, separate them with commas')
    ys: Optional[float] = Field(description='the value of yield strength, convert to MPa if the unit is not MPa, e.g. 1 GPa -> 1000 MPa')
    uts: Optional[float] = Field(description='the value of ultimate tensile strength, convert to MPa if the unit is not MPa, e.g. 1GPa -> 1000 MPa')
    elongation: Optional[float] = Field(description='the value of elongation, convert to percentage if the unit is not percentage, e.g. 1%')
    fabrication: Optional[str] = Field(description='The fabrication method of the alloy, e.g. vacuum arc-melting')
    thermal_mechanical_processings: Optional[str]  = Field(description='The sequential post-processing steps of the alloy separated by vertical bar "|", be briefly, eg., annealed at 900 °C for 4 h | homogenized at 1200 °C for 2 h')

    test_type: Literal['tensile', 'compressive']
    test_temperature: str = Field(description='The temperature at which the mechanical properties were tested, e.g. 25 °C. If not mentioned, use room temperature')

class Records(BaseModel):
    records: Optional[List[AlloyRecord]] = Field(description='The records of the alloy properties')

In [8]:
import langchain_openai
import langchain_core.prompts

model = langchain_openai.ChatOpenAI(model='gpt-4o', temperature=0.0)

template = langchain_core.prompts.ChatPromptTemplate(
    [
        ('system', """You are a domain expert of HEAs (high entropy alloys). You are provided with following fields:\n1. synthesis steps\n2. phases\n3. text or table description of tensile/compressive properties of HEAs.\n Give your response as user request."""),
        ('user', """Synthesis steps:\n{steps}\nPhases:\n{phases}\n{content}\nPlase extract the tensile or compressive properties along with correpond synthesis steps and phases from the text.\nNote: If the value are provided as average +/- standard deviation, use the average value, if the value are provied as minimum-maximum, use the minimum value.\nPrioritize the information in the table over text if there is a conflict."""),
    ]
)

In [12]:
from langchain_core.callbacks import BaseCallbackHandler


class LoggingHandler(BaseCallbackHandler):
    def on_chain_end(self, outputs, **kwargs) -> None:
        print(f"Chain ended, outputs: {outputs}")


callbacks = [LoggingHandler()]

In [16]:
from langchain_core.documents import Document

doc1 = Document(page_content='', metadata={'source': 1})
doc2 = Document(page_content='', metadata={'source': 1})
set([doc1, doc2])

TypeError: unhashable type: 'Document'

In [33]:
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from sisyphus.chain.chain_elements import DocInfo

distill_extract_template = ChatPromptTemplate(
    [
        ('system', 'You are a domain expert of HEAs (high entropy alloys)...'),
        ('user', '{paper}\n\nInstruction:\n{instruction}')
    ]
)
distill_extract_chain = distill_extract_template | model.with_structured_output(Records, method='json_schema')
DISTILL_INSTRUCTION = ''

def get_meta_info(docs):
    metadata = docs[0].metadata
    source = metadata['source']
    title = metadata['title']
    doi = metadata['doi']
    return source, title, doi

def reorder_docs(ordered, docs):
    """reorder the retrieved documents.
    Note: this function can deal with duplication in the docs!"""
    with_order = []
    used_i = []
    for doc in docs:
        for o_doc, i in ordered:
            if i in used_i:
                continue
            if doc == o_doc:
                with_order.append((doc, i))
                used_i.append(i)
                break
    final = sorted(with_order, key=lambda x: x[1])
    return [el[0] for el in final]

def render_docs(docs, title):
    """render docs to nicely formatted paper look.
    Since the tables are the most information dense format, we put it at the tail"""
    tables = [doc for doc in docs if doc.metadata['sub_titles'] == 'table']
    paras = [doc for doc in docs if doc.metadata['sub_titles'] != 'table']

    previous_titles = []
    scratch_pad = [title]
    scratch_pad
    for para in paras:
        sub_titles = para.metadata['sub_titles'].split('/')
        title_to_write = [title for title in sub_titles if title not in previous_titles] 
        previous_titles = sub_titles
        rendered_text = '\n'.join(title_to_write + [para.page_content])
        rendered_text = '\n' + rendered_text
        scratch_pad.append(rendered_text)

    scratch_pad.append('\nMechanical property tables')
    for table in tables:
        scratch_pad.append('\n' + table.page_content)

    return '\n'.join(scratch_pad)

def distill_extract(docs):
    with_order = list(zip(docs, range(len(docs))))
    source, title, doi = get_meta_info(docs)
    # when there is only abstract available 
    if len(docs) == 1:
        if get_target_para(source, QUERY_MECHANICAL, res_pattern, classifier_mech, 'tensile/compressive with value'):
            return (docs[0], distill_extract_chain.invoke({'paper': docs[0].page_content, 'instruction': DISTILL_INSTRUCTION}).records)
        return
    
    tensile_docs = get_target_para(source, QUERY_MECHANICAL, res_pattern, classifier_mech, 'tensile/compressive with value')
    tensile_tables = get_tables(docs)
    if not (tensile_docs or tensile_tables):
        return # no tensile relevant text/tables found
    synthesis_docs = get_target_para(source, QUERY_SYN, syn_pattern, classifier_syn, 'synthesis')
    phase_docs = get_target_para(source, QUERY_PHASE, res_pattern, classifier_pha, 'characterization_phase')
    abstract_docs = [doc for doc in docs if doc.metadata['sub_titles'] == 'Abstract']
    disordered_docs = tensile_docs + tensile_tables + synthesis_docs + phase_docs + abstract_docs
    ordered_docs = reorder_docs(with_order, disordered_docs)
    paper = render_docs(ordered_docs, title)
    # for test
    return paper

    results = distill_extract_chain.invoke({'paper': paper, 'instruction': DISTILL_INSTRUCTION})
    doc = Document(page_content=paper, metadata={'source': source, 'doi': doi})
    return doc, results.records

def return_valid(t):
    if t is None:
        return
    doc, result = t
    if result:
        return [DocInfo(doc, result)]
    return


In [36]:
from sisyphus.utils.helper_functions import get_plain_articledb, get_create_resultdb
from sisyphus.chain.chain_elements import Filter

db = get_plain_articledb('heas_batch_1')
article_getter = Filter(db)
chain = article_getter + distill_extract
with ThreadPoolExecutor(5) as executor:
    papers = []
    for paper in tqdm(executor.map(chain.compose, articles[:5]), total=5):
        papers.append(paper)

100%|██████████| 5/5 [00:26<00:00,  5.22s/it]


In [37]:
with open('inpect_format.txt', 'w', encoding='utf-8') as file:
    content = '\n==============\n'.join(papers)
    file.write(content)
    

In [32]:
lm.inspect_history(10)





[2025-01-14T20:31:20.060289]

System message:

Your input fields are:
1. `paragraph` (str)

Your output fields are:
1. `reasoning` (str)
2. `topic` (Literal[tensile/compressive with value, tensile/compressive without value, characterization, others])

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## paragraph ## ]]
{paragraph}

[[ ## reasoning ## ]]
{reasoning}

[[ ## topic ## ]]
{topic}        # note: the value you produce must be one of: tensile/compressive with value; tensile/compressive without value; characterization; others

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        assign topic to paragraphs of HEAs(high entropy alloys) papers. The topics include tensile/compressive with value, tensile/compressive without value, characterization or others.
        Note: a qualified tensile/compressive with value paragraph should explicitly mention at least one value related to yield strength, ultim

In [13]:
from sisyphus.chain.chain_elements import DocInfo, Document
def extract(docs):
    chain = template | model.with_structured_output(Records, method='json_schema')
    article = docs[0].metadata['source']
    title = docs[0].metadata['title']
    abstract = docs[0].metadata['abstract']
    doi = docs[0].metadata['doi']
    if len(docs) == 1:
        if get_target_para(docs, QUERY_MECHANICAL, res_pattern, classifier_mech, 'tensile/compressive with value'):
            return (docs[0], chain.invoke({
                'steps': '',
                'phases': '',
                'content': docs[0].page_content
            }).records)
        return
    mechanical = get_target_para(article, QUERY_MECHANICAL, res_pattern, classifier_mech, 'tensile/compressive with value')
    tables = get_tables(docs)
    if not (mechanical or tables):
        return # no mechanical properties found
    mechanical_text = '\n'.join([para.metadata['sub_titles'] + '\n' + para.page_content for para in mechanical])
    formatted_tables = '\n'.join(table.page_content for table in tables)
    formatted_mechanical = f'text:\n{mechanical_text}\ntables:\n{formatted_tables}'

    synthesis = get_target_para(article, QUERY_SYN, syn_pattern, classifier_syn, 'synthesis')
    phase = get_target_para(article, QUERY_PHASE, test_pattern, classifier_pha, 'characterization_phase')
    synthesis_steps, phases = '', ''
    if synthesis:
        formatted_syn_paras = '\n'.join([para.metadata['sub_titles'] + '\n' + para.page_content for para in synthesis])
        synthesis_steps = two_shot_steps_extractor(abstract=abstract, paragraph=formatted_syn_paras).steps
    if phase:
        formatted_phase_paras = '\n'.join([para.metadata['sub_titles'] + '\n' + para.page_content for para in phase])
        phases = phases_extractor(abstract=abstract, paragraph=formatted_phase_paras).phases

    results = chain.invoke(
        {
            'steps': synthesis_steps,
            'phases': phases,
            'content': formatted_mechanical
        }
    )
    document = Document(page_content=f'steps: {synthesis_steps}\nphases: {phases}\ncontent:\n{formatted_mechanical}', metadata={'source': article, 'title': title, 'abstract': abstract, 'doi': doi, 'type': 'correlation-extraction'})
    return (document, results.records)

def return_valid(t):
    if t is None:
        return
    doc, result = t
    if result:
        return [DocInfo(doc, result)]
    return
    

In [14]:
from sisyphus.utils.helper_functions import get_plain_articledb, get_create_resultdb
from sisyphus.chain.chain_elements import Filter, Writer

db = get_plain_articledb('300_heas')
result_db = get_create_resultdb('correlation_extraction_test', Records)
article_getter = Filter(db, with_abstract=True)
chain = article_getter + extract + return_valid + Writer(result_db)

/Users/pastalover/miniconda3/envs/sisyphus/lib/python3.10/site-packages/pydantic/main.py:1552: RuntimeWarning: fields may not start with an underscore, ignoring "__tablename__"
  warnings.warn(f'fields may not start with an underscore, ignoring "{f_name}"', RuntimeWarning)


In [15]:
result_db.clear_tables()

In [16]:
with ThreadPoolExecutor(5) as worker:
    futures = [worker.submit(chain.compose, article) for article in articles]
    for future in tqdm(as_completed(futures), total=len(articles)):
        future.result

100%|██████████| 20/20 [01:28<00:00,  4.44s/it]


In [43]:
json_ = result_db.load_as_json('gpt-4o for extraction, 4o-mini for classification', 'omit', db_name='correlation_extraction_test')

In [45]:
import json
with open('result.json', 'w') as f:
    json.dump(json_, f, indent=2)

In [20]:
get_target_para('10.1016&sol;j.jallcom.2019.04.121.html', QUERY_SYN, syn_pattern, classifier_syn, 'synthesis')

[Document(metadata={'doi': '10.1016/j.jallcom.2019.04.121', 'source': '10.1016&sol;j.jallcom.2019.04.121.html', 'sub_titles': '2 Materials and methods/2.1 Specimen preparation', 'title': 'Strengthening of a CrMnFeCoNi high-entropy alloy by carbide precipitation'}, page_content='The arc melting chamber containing the raw materials was evacuated to ~5 × 10-2 Pa, flushed twice with pure Ar with pump downs in between, and finally backfilled with Ar to a pressure of ~5 × 104 Pa. Then, a small piece of pure Ti was melted to getter any residual oxygen and the alloys were melted. The buttons were flipped and remelted five times to ensure chemical homogeneity before drop-casting into a rectangular copper mold measuring 10 × 17 × 70 mm3.'),
 Document(metadata={'doi': '10.1016/j.jallcom.2019.04.121', 'source': '10.1016&sol;j.jallcom.2019.04.121.html', 'sub_titles': '2 Materials and methods/2.1 Specimen preparation', 'title': 'Strengthening of a CrMnFeCoNi high-entropy alloy by carbide precipitati

In [18]:
lm.inspect_history(3)





[2025-01-14T11:32:25.122975]

System message:

Your input fields are:
1. `paragraph` (str)

Your output fields are:
1. `reasoning` (str)
2. `topic` (Literal[synthesis, characterization, others])

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## paragraph ## ]]
{paragraph}

[[ ## reasoning ## ]]
{reasoning}

[[ ## topic ## ]]
{topic}        # note: the value you produce must be one of: synthesis; characterization; others

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        assign topic to paragraphs of HEAs(high entropy alloys) papers. The topics include synthesis, characterization, and others.
        Note: a qualified synthesis paragraph should include the synthesis and processing of materials, including methods such as melting, casting, rolling, annealing, heat treatment.be very strict about your decision.


User message:

[[ ## paragraph ## ]]
The ingots were sealed in evacuated quartz tubes, ho

In [22]:
article_db.get('10.1016&sol;j.mtla.2024.102086.html')

[Document(metadata={'source': '10.1016&sol;j.mtla.2024.102086.html', 'doi': '10.1016/j.mtla.2024.102086', 'sub_titles': 'Abstract', 'title': 'Strengthening Al0.1CoCrFeNi high-entropy alloy via multiaxial cryogenic forging and low temperature annealing'}, page_content='Image, graphical abstract.'),
 Document(metadata={'source': '10.1016&sol;j.mtla.2024.102086.html', 'doi': '10.1016/j.mtla.2024.102086', 'sub_titles': 'table', 'title': 'Strengthening Al0.1CoCrFeNi high-entropy alloy via multiaxial cryogenic forging and low temperature annealing'}, page_content='Table 1 Mechanical properties of the FG, MACF-5 and LTA HEAs at room temperature.\nSpecimen name,Yield strength (MPa),Ultimate tensile strength (MPa),Elongation (%)\nFG,328,687,47.4\nMACF-5,985,1142,7.7\nLTA-673-1,1271,1368,6.5\nLTA-673-48,1240,1487,4.9'),
 Document(metadata={'source': '10.1016&sol;j.mtla.2024.102086.html', 'doi': '10.1016/j.mtla.2024.102086', 'sub_titles': 'table', 'title': 'Strengthening Al0.1CoCrFeNi high-entrop

In [30]:
template_ = langchain_core.prompts.ChatPromptTemplate(
    [
        ('system', 'extract heas tensile properties from text'),
        ('human', 'paper:\n{content}\nExtract the tensile properties of the HEAs material from the text')
    ]
)
docs = article_db.get('10.1002&sol;adem.201900587.html')
title = docs[0].metadata['title']
content = f'title: {title}\n' + '\n'.join([doc.page_content for doc in docs])
chain_ = template_ | model.with_structured_output(Records, method='json_schema')
result = chain_.invoke({'content': content})

In [31]:
result.records

[AlloyRecord(composition='V10Cr15Mn5Fe35Co10Ni25', phase='FCC', ys=430.0, uts=720.0, elongation=48.1, fabrication='vacuum induction melting', thermal_mechanical_processings='homogenized at 1100 °C for 6 h | cold rolled with 79% reduction | annealed at 900 °C for 10 min', test_type='tensile', test_temperature='room temperature'),
 AlloyRecord(composition='V10Cr15Mn5Fe35Co10Ni25', phase='FCC', ys=230.0, uts=532.0, elongation=57.6, fabrication='vacuum induction melting', thermal_mechanical_processings='homogenized at 1100 °C for 6 h | cold rolled with 79% reduction | annealed at 1100 °C for 60 min', test_type='tensile', test_temperature='room temperature'),
 AlloyRecord(composition='V10Cr15Mn5Fe35Co10Ni25', phase='FCC', ys=1120.0, uts=1447.0, elongation=15.9, fabrication='vacuum induction melting', thermal_mechanical_processings='homogenized at 1100 °C for 6 h | cold rolled with 79% reduction | annealed at 900 °C for 10 min | HPT 1/4 turn', test_type='tensile', test_temperature='room temp

In [35]:
content = """Effect of Initial Grain Size on Deformation Mechanism during High‐Pressure Torsion in V10Cr15Mn5Fe35Co10Ni25 High‐Entropy Alloy
doi: 10.1002/adem.201900587


Abstract
The transition of the deformation mechanism from the dislocation slip-mediated mechanism to the twin-mediated mechanism with increasing grain size is a well-observed phenomenon in materials with low stacking fault energy during compression/tensile tests. To understand this effect further at large strains, a V10Cr15Mn5Fe35Co10Ni25 (at%) high-entropy alloy with two initial average grain sizes is processed by high-pressure torsion (HPT) at different numbers of turns. The results indicate that initial grain size plays a significant role in the deformation mechanism during the HPT process. The fine-grained (FG) sample exhibits only a tangled dislocation structure, whereas mechanical twins are observed along with the formation of dislocations in the coarse-grained (CG) sample after the one-fourth turn. High dislocation density is observed in the CG sample after the one-fourth and first turn, which leads to a higher rate of hardness increment as compared with the FG sample. However, a similar microstructure and mechanical properties are observed after five turns of HPT processing in both FG and CG samples. After five turns, the microstructure consists of nanograins (average grain size ≈30 nm) with nanotwins, and the samples exhibit a very high ultimate tensile strength of ≈2 GPa with a reasonable elongation to failure of ≈6%.
2 Experimental Section
The HEA with a nominal composition of V10Cr15Mn5Fe35Co10Ni25 (at%) was fabricated using vacuum induction melting furnace using pure elements of V, Cr, Mn, Fe, Co, and Ni (purity >99.9%). The as-cast sample was subjected to homogenization heat treatment at 1100 °C for 6 h under an Ar atmosphere, followed by water quenching. The homogenized sample was cold rolled through multiple passes with a final rolling reduction ratio of ≈79% (from 6.2 to 1.3 mm). The disk-shaped samples (10 mm diameter) were prepared from the cold rolled sheet using electro-discharge machining. The disk samples were annealed at two different conditions (900 °C for 10 min and 1100 °C for 60 min) to obtain microstructure with fine grains and coarse grains, respectively. Finally, the HPT process was carried out on the annealed samples at different turns (N = 1/4, 1, and 5) using a pressure of 6 GPa and a rotation rate of 1 revolution per minute (rpm).
3 Results
3.2 Microstructural Evolution during HPT Processing
Figure shows the XRD patterns of fine-grained (FG) and coarse-grained (CG) samples before HPT processing (N = 0) and after five turns of HPT processing (N = 5). The XRD patterns of the annealed sample show a single FCC phase with no secondary phases, which confirms the thermodynamic calculations reported earlier for this HEA. The XRD patterns of HPT-processed samples also show a single FCC phase, indicating no evidence of phase transformation after HPT. Also, HPT processing leads to a significant peak broadening of XRD patterns, indicating grain refinement and lattice strain.
3.3 Mechanical Properties
The engineering stress versus strain curves of the FG and CG samples before and after HPT processing at different turns (1/4, 1, and 5 turns) are shown in Figure . The tensile curves indicate that samples in annealed condition exhibit low yield strength (YS) and high elongation to failure, whereas the HPT processing led to an enhancement in YS accompanied by a reduction in the elongation to failure. The strength-ductility trade-off is in agreement with the classic mechanical response of ultra-fine grained (UFG) metallic materials processed by SPD techniques. Table summarizes the relative data for the engineering YS, ultimate tensile strength (UTS), and total elongation to failure (δ) for both the FG and CG samples. The CG sample in annealed condition shows a YS of ≈230 MPa, and the FG sample shows a relatively higher YS of ≈430 MPa. The higher YS observed in the FG sample can be attributed to the Hall-Petch strengthening effect. The UTSs of the FG and CG samples are 751 and 532 MPa, respectively, and the CG sample presents slightly higher total elongation to failure in comparison with the FG sample (CG ≈58% and FG ≈48%).

Table 1. YS and UTS, and total elongation to failure (δ) of the FG and CG samples of V10Cr15Mn5Fe35Co10Ni25 HEA before (N = 0) and after HPT processing with increasing the number of turns
HPT Turns	Sample	YS (MPa)	UTS (MPa)	Elongation to Failure (%)
0	FG	430	720	48.1
0	CG	230	532	57.6
1/4	FG	1120	1447	15.9
1/4	CG	1270	1502	17.3
1	FG	1630	1813	12.9
1	CG	1660	1854	14.3
5	FG	1940	1986	6.0
5	CG	1950	2015	6.3

After 1/4 turn of HPT processing, the UTS increases significantly for both FG ( ≈ 1.4 GPa) and CG ( ≈ 1.5 GPa) samples. The strength increases further with increasing the number of turns, and the increase in strength is accompanied by a decrease in total elongation to failure. After five turns, the tensile strength of FG and CG samples reached similar values ( ≈ 2 GPa) with a total elongation to failure of ≈6%. The CG sample presents relatively higher YS and UTS as compared with the FG sample after 1/4 turn and 1 turn of HPT. This indicates that the strength enhancement in the CG sample is higher than that of the FG sample, considering the initial strength of the FG and CG samples (before HPT).
"""
result = chain_.invoke({'content': content})

In [34]:
result.records

[AlloyRecord(composition='V10Cr15Mn5Fe35Co10Ni25', phase='FCC', ys=230.0, uts=532.0, elongation=58.0, fabrication='vacuum induction melting', thermal_mechanical_processings='homogenized at 1100 °C for 6 h | cold rolled with 79% reduction | annealed at 1100 °C for 60 min', test_type='tensile', test_temperature='room temperature'),
 AlloyRecord(composition='V10Cr15Mn5Fe35Co10Ni25', phase='FCC', ys=430.0, uts=751.0, elongation=48.0, fabrication='vacuum induction melting', thermal_mechanical_processings='homogenized at 1100 °C for 6 h | cold rolled with 79% reduction | annealed at 900 °C for 10 min', test_type='tensile', test_temperature='room temperature'),
 AlloyRecord(composition='V10Cr15Mn5Fe35Co10Ni25', phase='FCC', ys=None, uts=1500.0, elongation=None, fabrication='vacuum induction melting', thermal_mechanical_processings='homogenized at 1100 °C for 6 h | cold rolled with 79% reduction | annealed at 1100 °C for 60 min | HPT 1/4 turn at 6 GPa', test_type='tensile', test_temperature='r

In [36]:
result.records

[AlloyRecord(composition='V10Cr15Mn5Fe35Co10Ni25', phase='FCC', ys=430.0, uts=720.0, elongation=48.1, fabrication='vacuum induction melting', thermal_mechanical_processings='homogenized at 1100 °C for 6 h | cold rolled with 79% reduction | annealed at 900 °C for 10 min', test_type='tensile', test_temperature='room temperature'),
 AlloyRecord(composition='V10Cr15Mn5Fe35Co10Ni25', phase='FCC', ys=230.0, uts=532.0, elongation=57.6, fabrication='vacuum induction melting', thermal_mechanical_processings='homogenized at 1100 °C for 6 h | cold rolled with 79% reduction | annealed at 1100 °C for 60 min', test_type='tensile', test_temperature='room temperature'),
 AlloyRecord(composition='V10Cr15Mn5Fe35Co10Ni25', phase='FCC', ys=1120.0, uts=1447.0, elongation=15.9, fabrication='vacuum induction melting', thermal_mechanical_processings='homogenized at 1100 °C for 6 h | cold rolled with 79% reduction | annealed at 900 °C for 10 min | HPT 1/4 turn at 6 GPa', test_type='tensile', test_temperature='